# Text Generation Referring to  Local Document with Transformers Pipeline, LangChain and Chroma

In this notebook, we will use a large language model (LLM) to perform text generation to answer questions based on local stored PDF document.

## Preparation

Install python packages.

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.7 MB/s eta 0:00:00


In [2]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.5 MB/s eta 0:00:00


In [3]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=fe3f339f3a234cb2c7c08887d5ba727cff1a40063a40b528bb4138fb3c290eb9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [4]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.7/966.7 kB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.2 MB/s

In [5]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 103.9 MB/s eta 0:00:00


In [6]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.0 MB/s eta 0:00:00


Check GPU information.

In [7]:
!nvidia-smi

Mon Jul 17 07:46:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Fix random number seed.

In [8]:
import random
import numpy as np
import torch

def set_seed(seed=0):
  # for Python
  random.seed(seed)

  # for NumPy
  np.random.seed(seed)

  # for PyTorch, CUDA and cuDNN
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [9]:
set_seed()

Get device.

In [10]:
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')
  device_id = 0
else:
  device = torch.device('cpu')
  device_id = -1

print(device)
print(device_id)

cuda
0


## Get PDF Document

As an example, we use the following PDF document describing the Manifesto for Agile Software Development by Information-technology Promotion Agency, Japan (IPA).
* https://www.ipa.go.jp/jinzai/skill-standard/plus-it-ui/itssplus/ps6vr70000001i7c-att/000065601.pdf

In [11]:
!wget https://www.ipa.go.jp/jinzai/skill-standard/plus-it-ui/itssplus/ps6vr70000001i7c-att/000065601.pdf

--2023-07-17 07:46:44--  https://www.ipa.go.jp/jinzai/skill-standard/plus-it-ui/itssplus/ps6vr70000001i7c-att/000065601.pdf
Resolving www.ipa.go.jp (www.ipa.go.jp)... 52.222.139.23, 52.222.139.33, 52.222.139.49, ...
Connecting to www.ipa.go.jp (www.ipa.go.jp)|52.222.139.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2019698 (1.9M) [application/pdf]
Saving to: ‘000065601.pdf’

000065601.pdf       100%[===================>]   1.93M  1.11MB/s    in 1.7s    

2023-07-17 07:46:47 (1.11 MB/s) - ‘000065601.pdf’ saved [2019698/2019698]



In [12]:
from langchain.document_loaders import PDFMinerLoader

file = '000065601.pdf'

loader = PDFMinerLoader(file)
text = loader.load()
pages = text[0].page_content.split('\x0c')

In [13]:
len(pages)

22

In [14]:
pages[0]

'アジャイル領域へのスキル変革の指針\n\nアジャイルソフトウェア開発宣言の\n読みとき方\n\n2020年2月\n\nITSS+ ／ アジャイル領域へのスキル変革の指針\n\nAll Rights Reserved Copyright© IPA 2018-2020\n\n'

## Construct Vector Database

Prepare an embedding.

Here, for example, we use OpenCALM-1B (https://huggingface.co/cyberagent/open-calm-1b), which was pre-trained on Japanese datasets by CyberAgent, Inc.

In [15]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

model_name = 'cyberagent/open-calm-1b'
embeddings = HuggingFaceEmbeddings(model_name=model_name)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/cyberagent_open-calm-1b were not used when initializing GPTNeoXModel: ['embed_out.weight']
- This IS expected if you are initializing GPTNeoXModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPTNeoXModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Convert to embedded vector for each page and store in vector DB.

In [16]:
from langchain.vectorstores import Chroma

vectordb = Chroma(embedding_function=embeddings, persist_directory='./db')

for i, page in enumerate(pages):
  if page == '':
    continue
  vectordb.add_texts(texts=[page],
                     metadatas=[{'source': file}],
                     ids=[f'id{i+1}'])

# Following code is needed to persist DB
# vectordb.persist()

## Building a Transformers Pipeline

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import pipeline

task='text-generation'

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

kwargs = {
    'max_new_tokens': 64,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.9,
    'repetition_penalty': 1.05,
    'pad_token_id': tokenizer.pad_token_id
}

pipe = pipeline(
    task=task,
    model=model,
    tokenizer=tokenizer,
    device=device_id,
    torch_dtype=torch.float16,
    **kwargs
)

## LangChain

In [18]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipe)

## Search the vector DB to extract candidates and pass them to LLMChain to get the answer

In [19]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = HuggingFacePipeline(pipeline=pipe)

template = """
質問: {query}

回答:
"""
prompt = PromptTemplate(template=template, input_variables=['query'])

llm_chain = LLMChain(llm=llm, prompt=prompt)

Search vector DB.

In [20]:
query = 'アジャイルソフトウェア開発宣言で、プロセスやツールよりも重視していることは？'
docs = vectordb.similarity_search(query=query, k=5)

Pass above search results to LLMChain to get answers.

In [21]:
llm_chain.run(input_documents=docs, query=query)

'アジャイルは「俊敏な」「型にはまらない」という2つの言葉で表現されますが、「柔軟性・流動性が重要だ。またその柔軟性を保証するための仕組みが重要である。」というのが答えです。「変化への対応は遅いかもしれないけれど,すぐに改善するんだ」。これがチームにとって大切なことです。”スピード”と簡単にいいますが'